# 脚本框架：

In [ ]:
start_date = "20150101"  # 设置全量计算时的开始时间
trade_date = "20240101"  # 每天的日期
common_param = {}  # 共有自定义参数

def DIY_FACTOR1_SCRIPT(trade_date, common_param, increment=True, start_date="20150101"):
    if increment == True:
        NEW_DATA_DF = calc_new_factor()
        # 你的因子增量计算代码
        return NEW_DATA_DF  # 返回trade_date这天新增的因子数据
    else:
        ALL_DATA_DF = calc_all_factor()
        # 你的因子全量计算代码
        return ALL_DATA_DF  # 返回截止到trade_date所有的因子数据
    # 返回一个df给数据库，有三列，分别是：trade_date、ts_code、factor_name

# 将所有的因子函数集中在一个dict中
fact_dict = {"FACTOR1": DIY_FACTOR1_SCRIPT, "FACTOR2": DIY_FACTOR2_SCRIPT}

# 测试用例框架：

In [31]:
# 生成测试日期
def get_datelist(num):
    import pandas as pd
    import numpy as np

    start_date = pd.to_datetime("2020-01-01")
    end_date = pd.to_datetime("2023-12-31")
    max_start_date = end_date - pd.Timedelta(days=num - 1)
    random_start = start_date + pd.to_timedelta(
        np.random.randint(0, (max_start_date - start_date).days + 1), unit="D"
    )
    date_range = pd.date_range(start=random_start, periods=num)
    return date_range

## 全量计算测试，测试连续的20天

In [ ]:
# 全量计算测试，测试连续的20天
start_date = "20150101"  # 设置全量计算时的开始时间

all_yes = True
for trade_date in get_datelist(20):
    common_param = make_common_param()  # 在这里准备你需要的数据和参数
    for x in fact_dict:
        try:
            df = fact_dict[x](
                trade_date=trade_date,
                common_param=common_param,
                increment=True,
                start_date=start_date,
            )
            if type(df) != pd.DataFrame:
                raise ValueError("返回值类型有误")
            if len(df) < 1:
                raise ValueError("返回值行数有误")
            if len(df.columns) != 3:
                raise ValueError("返回值列数有误")
            if df.columns[2] != x:
                raise ValueError("返回值第三列名字有误")
            if len(df) != len(df.dropna()):
                raise ValueError("返回值中含有None")
            print(f"{x}测试通过")
        except Exception as e:
            print(f"因子{x}在{trade_date}出错:{e}")
            all_yes = False
if all_yes:
    print("全量计算测试全部通过！")
else:
    print("没有全部通过全量计算测试！")

##  增量计算测试，测试连续的20天

In [ ]:
# 增量计算测试，测试连续的20天
all_yes = True
for trade_date in get_datelist(20):
    print(trade_date, "正在测试")
    common_param = make_common_param()  # 在这里准备你需要的数据和参数
    for x in fact_dict:
        try:
            df = fact_dict[x](
                trade_date=trade_date, common_param=common_param, increment=False
            )
            if type(df) == pd.DataFrame:
                if len(df) < 1:
                    raise ValueError("返回值行数有误")
                if len(df.columns) != 3:
                    raise ValueError("返回值列数有误")
                if df.columns[2] != x:
                    raise ValueError("返回值第三列名字有误")
                if len(df) != len(df.dropna()):
                    raise ValueError("返回值中含有None")
        except Exception as e:
            print(f"因子{x}在{trade_date}出错:{e}")
            all_yes = False
if all_yes:
    print("增量计算测试全部通过！")
else:
    print("没有全部通过增量计算测试！")